In [51]:
import json
import numpy as np
import pandas as pd
import os

from math import cos, sin, exp, sqrt, pi, e

from src.OspitalettoDataset import OspitalettoDataset
from src.NOAA2010Dataset import NOAA2010Dataset
from src.InsPireDataset import InsPireDataset

## Constants to be used across the notebook

In [8]:
load_or_write_new = int(input("Press 1 if you want to reload previous simulation values"))

if load_or_write_new == 1:
    simulation_values_path = os.path.join(".", "data", "simulation_values.json")
    
    with open(simulation_values_path, "r") as f:
        values = json.load(f)
    
    Tmin_i = values["Tmin_i"]
    Tmax_i = values["Tmax_i"]

    Tdhw = values["Tdhw"]

    Ts1 = values["Ts1"]
    Ts2 = values["Ts2"]

    DT_evap = values["DT_evap"]
    
    n_buildings_SFH=values["n_buildings_SFH"]
    n_buildings_MFH=values["n_buildings_MFH"]
    
else:
    # Min and Max temperatures to be received by the user.
    # Values [40,...,60]
    Tmin_i = float(input("Enter min setpoint temperature: "))
    Tmax_i = float(input("Enter max setpoint temperature: "))

    # Domestic hot water temperature.
    # Values [45,...,55]
    Tdhw = float(input("Enter domestic hot water temperature: "))

    # Our model supposes that the system has two different sources of heat.
    # Values [20,...,30]
    Ts1 = float(input("Temperature of source 1: "))
    Ts2 = float(input("Temperature of source 2: "))

    # difference in temperature between the supply (system->heat->house) and return (house -> heat -> system) pipes
    # Values [1,...,10]
    DT_evap = float(input("Temperature difference among the supply and return pipes: "))
    
    # For the data visualization course, we will select 0 SFH & 1600 MFH buildings, which will cover an area of 1 km2
    n_buildings_SFH =float(input("Enter the number of SFH buildings: #"))
    n_buildings_MFH=float(input("Enter the number of MFH buildings:#"))
    
print(f"* {Tmin_i=}\n"
      f"* {Tmax_i=}\n"
      f"* {Tdhw=}\n"
      f"* {Ts1=}\n"
      f"* {Ts2=}\n"
      f"* {DT_evap=}\n"
      f"* {n_buildings_SFH=}\n"
      f"* {n_buildings_MFH=}\n"
     )

Press 1 if you want to reload previous simulation values 1


* Tmin_i=40.0
* Tmax_i=55.0
* Tdhw=48.0
* Ts1=23.0
* Ts2=25.0
* DT_evap=10.0
* n_buildings_SFH=0
* n_buildings_MFH=1600



# Load Sources working hours

The file must be a boolean matrix $W$ of $24 x 7$. The rows represent the hours in a day, and the columns represent the days in the week. For all $i \in \{0,...,23\}$ and $j \in \{0,..., 6\}$ we have that $w_{i,j} \in \{0, 1\}$, where $w_{i,j} = 0$ means that the source doesn't produce energy at that hour $i$ on that day $j$. Similarly, $w_{i,j} = 1$ means that the source produces energy at that hour $i$ on that day $j$.

In [9]:
s1_schedule_path = os.path.join(".", "data", "private", "s1_source_fake_schedule.xlsx")
if load_or_write_new == 1:
    s1_schedule = np.array(values["s1_schedule"])
else:    
    s1_schedule = pd.read_excel(s1_schedule_path, index_col="Time").to_numpy()

print(f"* {s1_schedule_path=}\n"
      f"* {s1_schedule=}\n"
      f"* {s1_schedule[6,1]=}\n" # Tuesdays are closed at 6 (equal to 0)
      f"* {s1_schedule[7,1]=}\n") # but open are 7 (equal to 1)

* s1_schedule_path='./data/private/s1_source_fake_schedule.xlsx'
* s1_schedule=array([[0, 0, 1, 1, 1, 1, 0],
       [0, 0, 1, 1, 1, 1, 0],
       [0, 0, 1, 1, 1, 1, 0],
       [0, 0, 1, 1, 1, 1, 0],
       [0, 0, 1, 1, 1, 1, 0],
       [0, 0, 1, 1, 1, 1, 0],
       [0, 0, 1, 1, 1, 1, 0],
       [0, 1, 1, 1, 1, 1, 0],
       [0, 1, 1, 1, 1, 1, 0],
       [0, 1, 1, 1, 1, 1, 0],
       [0, 1, 1, 1, 1, 1, 0],
       [0, 1, 1, 1, 1, 1, 0],
       [0, 1, 1, 1, 1, 1, 0],
       [0, 1, 1, 1, 1, 1, 0],
       [0, 1, 1, 1, 1, 1, 0],
       [0, 1, 1, 1, 1, 1, 0],
       [0, 1, 1, 1, 1, 0, 0],
       [0, 1, 1, 1, 1, 0, 0],
       [0, 1, 1, 1, 1, 0, 0],
       [0, 1, 1, 1, 1, 0, 0],
       [0, 1, 1, 1, 1, 0, 0],
       [0, 1, 1, 1, 1, 0, 0],
       [0, 1, 1, 1, 1, 0, 0],
       [0, 1, 1, 1, 1, 0, 0]])
* s1_schedule[6,1]=0
* s1_schedule[7,1]=1



In [10]:
s2_schedule_path = os.path.join(".", "data", "private", "s2_source_fake_schedule.xlsx")
if load_or_write_new == 1:
    s2_schedule = np.array(values["s2_schedule"])
else: 
    s2_schedule = pd.read_excel(s2_schedule_path, index_col="Time").to_numpy()

print(f"* {s2_schedule_path=}\n"
      f"* {s2_schedule=}\n"
      f"* {s2_schedule[15,1]=}\n" # Tuesdays are closed at 15 (equal to 0)
      f"* {s2_schedule[16,1]=}\n") # but open are 16 (equal to 1)

* s2_schedule_path='./data/private/s2_source_fake_schedule.xlsx'
* s2_schedule=array([[1, 1, 0, 0, 1, 1, 1],
       [1, 1, 0, 0, 1, 1, 1],
       [1, 1, 0, 0, 1, 1, 1],
       [1, 1, 0, 0, 1, 1, 1],
       [1, 1, 0, 0, 1, 1, 1],
       [1, 1, 0, 0, 1, 1, 1],
       [1, 1, 0, 0, 1, 1, 1],
       [1, 1, 0, 1, 1, 1, 1],
       [1, 1, 0, 1, 1, 1, 1],
       [1, 1, 0, 1, 1, 1, 1],
       [1, 1, 0, 1, 1, 1, 1],
       [1, 1, 0, 1, 1, 1, 1],
       [1, 1, 0, 1, 1, 1, 1],
       [1, 1, 0, 1, 1, 1, 1],
       [1, 1, 0, 1, 1, 1, 1],
       [1, 1, 0, 1, 1, 1, 1],
       [1, 0, 0, 1, 1, 1, 1],
       [1, 0, 0, 1, 1, 1, 1],
       [1, 0, 0, 1, 1, 1, 1],
       [1, 0, 0, 1, 1, 1, 1],
       [1, 0, 0, 1, 1, 1, 1],
       [1, 0, 0, 1, 1, 1, 1],
       [1, 0, 0, 1, 1, 1, 1],
       [1, 0, 0, 1, 1, 1, 1]])
* s2_schedule[15,1]=1
* s2_schedule[16,1]=0



## Take note of the values for the current simulation

In [11]:
simulation_values = {
    "Tmin_i": Tmin_i,
    "Tmax_i": Tmax_i,
    "Tdhw": Tdhw,
    "Ts1": Ts1,
    "Ts2": Ts2,
    "DT_evap": DT_evap,
    "s1_schedule": s1_schedule.tolist(),
    "s2_schedule": s2_schedule.tolist(),
}
simulation_values

{'Tmin_i': 40.0,
 'Tmax_i': 55.0,
 'Tdhw': 48.0,
 'Ts1': 23.0,
 'Ts2': 25.0,
 'DT_evap': 10.0,
 's1_schedule': [[0, 0, 1, 1, 1, 1, 0],
  [0, 0, 1, 1, 1, 1, 0],
  [0, 0, 1, 1, 1, 1, 0],
  [0, 0, 1, 1, 1, 1, 0],
  [0, 0, 1, 1, 1, 1, 0],
  [0, 0, 1, 1, 1, 1, 0],
  [0, 0, 1, 1, 1, 1, 0],
  [0, 1, 1, 1, 1, 1, 0],
  [0, 1, 1, 1, 1, 1, 0],
  [0, 1, 1, 1, 1, 1, 0],
  [0, 1, 1, 1, 1, 1, 0],
  [0, 1, 1, 1, 1, 1, 0],
  [0, 1, 1, 1, 1, 1, 0],
  [0, 1, 1, 1, 1, 1, 0],
  [0, 1, 1, 1, 1, 1, 0],
  [0, 1, 1, 1, 1, 1, 0],
  [0, 1, 1, 1, 1, 0, 0],
  [0, 1, 1, 1, 1, 0, 0],
  [0, 1, 1, 1, 1, 0, 0],
  [0, 1, 1, 1, 1, 0, 0],
  [0, 1, 1, 1, 1, 0, 0],
  [0, 1, 1, 1, 1, 0, 0],
  [0, 1, 1, 1, 1, 0, 0],
  [0, 1, 1, 1, 1, 0, 0]],
 's2_schedule': [[1, 1, 0, 0, 1, 1, 1],
  [1, 1, 0, 0, 1, 1, 1],
  [1, 1, 0, 0, 1, 1, 1],
  [1, 1, 0, 0, 1, 1, 1],
  [1, 1, 0, 0, 1, 1, 1],
  [1, 1, 0, 0, 1, 1, 1],
  [1, 1, 0, 0, 1, 1, 1],
  [1, 1, 0, 1, 1, 1, 1],
  [1, 1, 0, 1, 1, 1, 1],
  [1, 1, 0, 1, 1, 1, 1],
  [1, 1, 0, 1, 1, 1, 1],

## Read the dataset

In [12]:
ospitaletto2019 = OspitalettoDataset()
noaa2010Dataset = NOAA2010Dataset()
insPireDataset = InsPireDataset()

AVAILABLE_DATASETS = dict()
AVAILABLE_DATASETS.update(ospitaletto2019.load_processed_data())
AVAILABLE_DATASETS.update(noaa2010Dataset.load_processed_data())
AVAILABLE_DATASETS.update(insPireDataset.load_processed_data())
AVAILABLE_DATASETS.keys()

dict_keys(['ospitaletto', 'miami_florida', 'fresno_california', 'olympia_washington', 'rochester_newyork', 'london_uk', 'madrid_spain', 'rome_italy', 'stuttgart_germany'])

In [107]:
dataset_to_work = "rochester_newyork"
dataset_to_work

'rochester_newyork'

In [108]:
Tamb_hourly = AVAILABLE_DATASETS[dataset_to_work]
Tamb_hourly

,air_temp,dayofyear,month,hourofyear,air_temp_fit,ground_temp,aquifer_temp,space_heating_temp,hot_water_temp,user_temp,...,E_loss_tot,city_key,climate,DHW_cons,SH_cons,SFH_bldg_tot,MFH_bldg_tot,%SH_y,%DHW_y,DHW_hourly_consumption_ratio
timestamp,,,,,,,,,,,,,,,,,,,,,
2010-01-01 01:00:00,-2.9,1,1,2,-2.749240,2.752620,9.187559,55.0,48.0,54.86,...,0.064990,rochester_newyork,Continental,21.7,68.9,90.6,90.6,0.760486,0.239514,0.003885
2010-01-01 02:00:00,-3.1,1,1,3,-2.752504,2.748357,9.187560,55.0,48.0,54.86,...,0.065007,rochester_newyork,Continental,21.7,68.9,90.6,90.6,0.760486,0.239514,0.003885
2010-01-01 03:00:00,-3.2,1,1,4,-2.755763,2.744097,9.187560,55.0,48.0,54.86,...,0.065024,rochester_newyork,Continental,21.7,68.9,90.6,90.6,0.760486,0.239514,0.003885
2010-01-01 04:00:00,-3.3,1,1,5,-2.759015,2.739840,9.187560,55.0,48.0,54.86,...,0.065041,rochester_newyork,Continental,21.7,68.9,90.6,90.6,0.760486,0.239514,0.003885
2010-01-01 05:00:00,-3.3,1,1,6,-2.762261,2.735587,9.187560,55.0,48.0,54.86,...,0.065058,rochester_newyork,Continental,21.7,68.9,90.6,90.6,0.760486,0.239514,0.003885
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2010-12-31 19:00:00,-1.9,365,12,8756,-2.732825,2.778268,9.187559,55.0,48.0,54.79,...,0.064887,rochester_newyork,Continental,21.7,68.9,90.6,90.6,0.760486,0.239514,0.053699
2010-12-31 20:00:00,-2.2,365,12,8757,-2.736120,2.773985,9.187559,55.0,48.0,54.79,...,0.064904,rochester_newyork,Continental,21.7,68.9,90.6,90.6,0.760486,0.239514,0.049895
2010-12-31 21:00:00,-2.3,365,12,8758,-2.739409,2.769705,9.187559,55.0,48.0,54.79,...,0.064921,rochester_newyork,Continental,21.7,68.9,90.6,90.6,0.760486,0.239514,0.006780


## Space Heating and Domestic Hot Water consumption

The user determines the number of buildings and building type (single family house or small-multi family house) supplied by a district heating network. Data from the Inspire project was used to estimate the annual energy consumption for each building typology in kWh/m2-y. SFHs and MFHs correspond to reference buildings in each climatic zone built between 1945-1970, non-refurbished. 

The SFH and MFH models have a fixed geometry for all the climates and periods of construction. It has been defined following the common characteristics for a European buildings. The SFH building model is composed of two storeys with a total of 100 m² of living area (Inspire project, D2.1c(2014)). The small-MFH correspond to a reference residential building that has two dwellings per floor, with a total floor area of 500 m2.

### Residential building typologies: SFH and MFH

#### Single-Family units

<img src="./images/SFH.png">        





#### Multi-Families units

<img src="./images/MFH.png">




### District heating system diagram

<img src="./images/DHC_system.png">

### Calculating SH, DWH and Heat Demand

This calculation is based on the number of SFH ($ |SFH| $) and MFH ($ |MFH| $) units. It also depends on the total heat needed by SFH ($ TOT_{SFH} $) and MFH ($ TOT_{MFH} $) units. The formula for the Heat Demand($ HD $) is the following  

\begin{align*}
HD = |SFH| * 100 * TOT_{SFH} + |MFH| * 500 * TOT_{MFH}
\end{align*}

It's important to stress that this calculation is done on a year basis.

In [109]:
Tamb_hourly["Heat_demand"] = (n_buildings_SFH * 100 * Tamb_hourly["SFH_bldg_tot"]) + (n_buildings_MFH * 500 * Tamb_hourly["MFH_bldg_tot"])
Tamb_hourly["SH_demand"] = Tamb_hourly["Heat_demand"] * Tamb_hourly["%SH_y"]
Tamb_hourly["DHW_demand"] = Tamb_hourly["Heat_demand"] * Tamb_hourly["%DHW_y"] # Equivalently, we can change Tamb_hourly["%DHW_y"] with (1 - Tamb_hourly["%SH_y"])
Tamb_hourly

,air_temp,dayofyear,month,hourofyear,air_temp_fit,ground_temp,aquifer_temp,space_heating_temp,hot_water_temp,user_temp,...,DHW_cons,SH_cons,SFH_bldg_tot,MFH_bldg_tot,%SH_y,%DHW_y,DHW_hourly_consumption_ratio,Heat_demand,SH_demand,DHW_demand
timestamp,,,,,,,,,,,,,,,,,,,,,
2010-01-01 01:00:00,-2.9,1,1,2,-2.749240,2.752620,9.187559,55.0,48.0,54.86,...,21.7,68.9,90.6,90.6,0.760486,0.239514,0.003885,7.248e+07,5.512e+07,1.736e+07
2010-01-01 02:00:00,-3.1,1,1,3,-2.752504,2.748357,9.187560,55.0,48.0,54.86,...,21.7,68.9,90.6,90.6,0.760486,0.239514,0.003885,7.248e+07,5.512e+07,1.736e+07
2010-01-01 03:00:00,-3.2,1,1,4,-2.755763,2.744097,9.187560,55.0,48.0,54.86,...,21.7,68.9,90.6,90.6,0.760486,0.239514,0.003885,7.248e+07,5.512e+07,1.736e+07
2010-01-01 04:00:00,-3.3,1,1,5,-2.759015,2.739840,9.187560,55.0,48.0,54.86,...,21.7,68.9,90.6,90.6,0.760486,0.239514,0.003885,7.248e+07,5.512e+07,1.736e+07
2010-01-01 05:00:00,-3.3,1,1,6,-2.762261,2.735587,9.187560,55.0,48.0,54.86,...,21.7,68.9,90.6,90.6,0.760486,0.239514,0.003885,7.248e+07,5.512e+07,1.736e+07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2010-12-31 19:00:00,-1.9,365,12,8756,-2.732825,2.778268,9.187559,55.0,48.0,54.79,...,21.7,68.9,90.6,90.6,0.760486,0.239514,0.053699,7.248e+07,5.512e+07,1.736e+07
2010-12-31 20:00:00,-2.2,365,12,8757,-2.736120,2.773985,9.187559,55.0,48.0,54.79,...,21.7,68.9,90.6,90.6,0.760486,0.239514,0.049895,7.248e+07,5.512e+07,1.736e+07
2010-12-31 21:00:00,-2.3,365,12,8758,-2.739409,2.769705,9.187559,55.0,48.0,54.79,...,21.7,68.9,90.6,90.6,0.760486,0.239514,0.006780,7.248e+07,5.512e+07,1.736e+07


### Domestic hot water consumption

Domestic hot water consumption is not weather influenced, and its variation is almost constant over the year. The hourly demand for hot water is estimated by the percentage of the total heat demand ($ HD_y $) is hot water $ DHW_y $. This data is obtained from the INSPIRE database for different climatic zones. In this particular case, the climatic zones and reference cities considered are:

* Stuttgart, Germany for Continental climate
* London, United Kingdom for Oceanic climate
* Madrid, Spain for Southern-dry climate
* Rome, Italy for Mediterranean climate

The total domestic hot water demand is evenly distributed over each day of the year, and then its hourly distribution is obtained by multiplying the daily needs by an hourly random profile. 

\begin{align*}
DHW_{d} = \frac{HD_{y}*DHW_{\%y}}{d}
\end{align*}

The DHW consumption is calculated in an hourly fashion.

In [110]:
Tamb_hourly["DHW_consumption"] = Tamb_hourly["DHW_demand"] * Tamb_hourly["DHW_hourly_consumption_ratio"] / 365
Tamb_hourly

,air_temp,dayofyear,month,hourofyear,air_temp_fit,ground_temp,aquifer_temp,space_heating_temp,hot_water_temp,user_temp,...,SH_cons,SFH_bldg_tot,MFH_bldg_tot,%SH_y,%DHW_y,DHW_hourly_consumption_ratio,Heat_demand,SH_demand,DHW_demand,DHW_consumption
timestamp,,,,,,,,,,,,,,,,,,,,,
2010-01-01 01:00:00,-2.9,1,1,2,-2.749240,2.752620,9.187559,55.0,48.0,54.86,...,68.9,90.6,90.6,0.760486,0.239514,0.003885,7.248e+07,5.512e+07,1.736e+07,184.777
2010-01-01 02:00:00,-3.1,1,1,3,-2.752504,2.748357,9.187560,55.0,48.0,54.86,...,68.9,90.6,90.6,0.760486,0.239514,0.003885,7.248e+07,5.512e+07,1.736e+07,184.777
2010-01-01 03:00:00,-3.2,1,1,4,-2.755763,2.744097,9.187560,55.0,48.0,54.86,...,68.9,90.6,90.6,0.760486,0.239514,0.003885,7.248e+07,5.512e+07,1.736e+07,184.777
2010-01-01 04:00:00,-3.3,1,1,5,-2.759015,2.739840,9.187560,55.0,48.0,54.86,...,68.9,90.6,90.6,0.760486,0.239514,0.003885,7.248e+07,5.512e+07,1.736e+07,184.777
2010-01-01 05:00:00,-3.3,1,1,6,-2.762261,2.735587,9.187560,55.0,48.0,54.86,...,68.9,90.6,90.6,0.760486,0.239514,0.003885,7.248e+07,5.512e+07,1.736e+07,184.777
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2010-12-31 19:00:00,-1.9,365,12,8756,-2.732825,2.778268,9.187559,55.0,48.0,54.79,...,68.9,90.6,90.6,0.760486,0.239514,0.053699,7.248e+07,5.512e+07,1.736e+07,2554
2010-12-31 20:00:00,-2.2,365,12,8757,-2.736120,2.773985,9.187559,55.0,48.0,54.79,...,68.9,90.6,90.6,0.760486,0.239514,0.049895,7.248e+07,5.512e+07,1.736e+07,2373.1
2010-12-31 21:00:00,-2.3,365,12,8758,-2.739409,2.769705,9.187559,55.0,48.0,54.79,...,68.9,90.6,90.6,0.760486,0.239514,0.006780,7.248e+07,5.512e+07,1.736e+07,322.489


### Space heating degree

The amount (in degrees) and for how long (in hours) of thermal heat required to keep the indoor building temperature at a comfortable level will vary depending on different climates. The base temperature selected was set to 15 °C.

The space heating hourly profile for each location is then retrieved with a time dependency according to the heating degree method, known as the "Integration method".

More info on calculation method: https://www.degreedays.net/introduction

In [111]:
def heating_degree(T_amb):
    T_base = 15
    return max(0, T_base - T_amb)

In [112]:
Tamb_hourly["heating_degree_days"] = Tamb_hourly.air_temp.apply(heating_degree)
Tamb_hourly["space_heating_dist"] = Tamb_hourly.heating_degree_days / max(sum(Tamb_hourly.heating_degree_days), 1)
Tamb_hourly["SH_consumption"] = Tamb_hourly["SH_demand"] * Tamb_hourly["space_heating_dist"]
Tamb_hourly

,air_temp,dayofyear,month,hourofyear,air_temp_fit,ground_temp,aquifer_temp,space_heating_temp,hot_water_temp,user_temp,...,%SH_y,%DHW_y,DHW_hourly_consumption_ratio,Heat_demand,SH_demand,DHW_demand,DHW_consumption,heating_degree_days,space_heating_dist,SH_consumption
timestamp,,,,,,,,,,,,,,,,,,,,,
2010-01-01 01:00:00,-2.9,1,1,2,-2.749240,2.752620,9.187559,55.0,48.0,54.86,...,0.760486,0.239514,0.003885,7.248e+07,5.512e+07,1.736e+07,184.777,17.9,0.000273,15023.9
2010-01-01 02:00:00,-3.1,1,1,3,-2.752504,2.748357,9.187560,55.0,48.0,54.86,...,0.760486,0.239514,0.003885,7.248e+07,5.512e+07,1.736e+07,184.777,18.1,0.000276,15191.7
2010-01-01 03:00:00,-3.2,1,1,4,-2.755763,2.744097,9.187560,55.0,48.0,54.86,...,0.760486,0.239514,0.003885,7.248e+07,5.512e+07,1.736e+07,184.777,18.2,0.000277,15275.7
2010-01-01 04:00:00,-3.3,1,1,5,-2.759015,2.739840,9.187560,55.0,48.0,54.86,...,0.760486,0.239514,0.003885,7.248e+07,5.512e+07,1.736e+07,184.777,18.3,0.000279,15359.6
2010-01-01 05:00:00,-3.3,1,1,6,-2.762261,2.735587,9.187560,55.0,48.0,54.86,...,0.760486,0.239514,0.003885,7.248e+07,5.512e+07,1.736e+07,184.777,18.3,0.000279,15359.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2010-12-31 19:00:00,-1.9,365,12,8756,-2.732825,2.778268,9.187559,55.0,48.0,54.79,...,0.760486,0.239514,0.053699,7.248e+07,5.512e+07,1.736e+07,2554,16.9,0.000257,14184.6
2010-12-31 20:00:00,-2.2,365,12,8757,-2.736120,2.773985,9.187559,55.0,48.0,54.79,...,0.760486,0.239514,0.049895,7.248e+07,5.512e+07,1.736e+07,2373.1,17.2,0.000262,14436.4
2010-12-31 21:00:00,-2.3,365,12,8758,-2.739409,2.769705,9.187559,55.0,48.0,54.79,...,0.760486,0.239514,0.006780,7.248e+07,5.512e+07,1.736e+07,322.489,17.3,0.000263,14520.3


### Total hourly consumption

This is as easy as 

\begin{align*}
CON_{TOT} = CON_{SH} + CON_{DHW}
\end{align*}



In [113]:
Tamb_hourly["Total_consumption"] = Tamb_hourly["DHW_consumption"] + Tamb_hourly["SH_consumption"]
Tamb_hourly

,air_temp,dayofyear,month,hourofyear,air_temp_fit,ground_temp,aquifer_temp,space_heating_temp,hot_water_temp,user_temp,...,%DHW_y,DHW_hourly_consumption_ratio,Heat_demand,SH_demand,DHW_demand,DHW_consumption,heating_degree_days,space_heating_dist,SH_consumption,Total_consumption
timestamp,,,,,,,,,,,,,,,,,,,,,
2010-01-01 01:00:00,-2.9,1,1,2,-2.749240,2.752620,9.187559,55.0,48.0,54.86,...,0.239514,0.003885,7.248e+07,5.512e+07,1.736e+07,184.777,17.9,0.000273,15023.9,15208.7
2010-01-01 02:00:00,-3.1,1,1,3,-2.752504,2.748357,9.187560,55.0,48.0,54.86,...,0.239514,0.003885,7.248e+07,5.512e+07,1.736e+07,184.777,18.1,0.000276,15191.7,15376.5
2010-01-01 03:00:00,-3.2,1,1,4,-2.755763,2.744097,9.187560,55.0,48.0,54.86,...,0.239514,0.003885,7.248e+07,5.512e+07,1.736e+07,184.777,18.2,0.000277,15275.7,15460.5
2010-01-01 04:00:00,-3.3,1,1,5,-2.759015,2.739840,9.187560,55.0,48.0,54.86,...,0.239514,0.003885,7.248e+07,5.512e+07,1.736e+07,184.777,18.3,0.000279,15359.6,15544.4
2010-01-01 05:00:00,-3.3,1,1,6,-2.762261,2.735587,9.187560,55.0,48.0,54.86,...,0.239514,0.003885,7.248e+07,5.512e+07,1.736e+07,184.777,18.3,0.000279,15359.6,15544.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2010-12-31 19:00:00,-1.9,365,12,8756,-2.732825,2.778268,9.187559,55.0,48.0,54.79,...,0.239514,0.053699,7.248e+07,5.512e+07,1.736e+07,2554,16.9,0.000257,14184.6,16738.6
2010-12-31 20:00:00,-2.2,365,12,8757,-2.736120,2.773985,9.187559,55.0,48.0,54.79,...,0.239514,0.049895,7.248e+07,5.512e+07,1.736e+07,2373.1,17.2,0.000262,14436.4,16809.4
2010-12-31 21:00:00,-2.3,365,12,8758,-2.739409,2.769705,9.187559,55.0,48.0,54.79,...,0.239514,0.006780,7.248e+07,5.512e+07,1.736e+07,322.489,17.3,0.000263,14520.3,14842.8


In [114]:
Tamb_hourly.columns

Index(['air_temp', 'dayofyear', 'month', 'hourofyear', 'air_temp_fit',
       'ground_temp', 'aquifer_temp', 'space_heating_temp', 'hot_water_temp',
       'user_temp', 'source1_temp', 'source2_temp', 'net_temp', 'COP',
       'E_loss_s', 'E_loss_r', 'E_loss_tot', 'city_key', 'climate', 'DHW_cons',
       'SH_cons', 'SFH_bldg_tot', 'MFH_bldg_tot', '%SH_y', '%DHW_y',
       'DHW_hourly_consumption_ratio', 'Heat_demand', 'SH_demand',
       'DHW_demand', 'DHW_consumption', 'heating_degree_days',
       'space_heating_dist', 'SH_consumption', 'Total_consumption'],
      dtype='object')

In [115]:
Tamb_hourly["SH_consumption"].shape

(8759,)



## Calculate Space Heating and Hot Water Temperature

The space Heating is calculated using a climatic curve. The Hot Water temperature is constant across the year.

In [116]:
def climatic_curve(Tamb_h):
    Tmin_o = 2.38 #minimum outdoor T threshold in which the space heating system turns on
    Tmax_o = 7.25 #maximum outdoor T threshold in which the space heating system turns off

    if Tamb_h <= Tmin_o:   
        Tsh = Tmax_i
    elif Tamb_h >= Tmax_o:
        Tsh = Tmin_i
    else:
        m = (Tmax_i-Tmin_i)/(Tmin_o-Tmax_o)
        b = -m*Tmin_o+Tmax_i
        Tsh = m*Tamb_h+b
    return Tsh
 

In [117]:
Tamb_hourly['space_heating_temp'] = Tamb_hourly.air_temp.apply(climatic_curve)
Tamb_hourly["hot_water_temp"] = Tdhw
Tamb_hourly

,air_temp,dayofyear,month,hourofyear,air_temp_fit,ground_temp,aquifer_temp,space_heating_temp,hot_water_temp,user_temp,...,%DHW_y,DHW_hourly_consumption_ratio,Heat_demand,SH_demand,DHW_demand,DHW_consumption,heating_degree_days,space_heating_dist,SH_consumption,Total_consumption
timestamp,,,,,,,,,,,,,,,,,,,,,
2010-01-01 01:00:00,-2.9,1,1,2,-2.749240,2.752620,9.187559,55.0,48.0,54.86,...,0.239514,0.003885,7.248e+07,5.512e+07,1.736e+07,184.777,17.9,0.000273,15023.9,15208.7
2010-01-01 02:00:00,-3.1,1,1,3,-2.752504,2.748357,9.187560,55.0,48.0,54.86,...,0.239514,0.003885,7.248e+07,5.512e+07,1.736e+07,184.777,18.1,0.000276,15191.7,15376.5
2010-01-01 03:00:00,-3.2,1,1,4,-2.755763,2.744097,9.187560,55.0,48.0,54.86,...,0.239514,0.003885,7.248e+07,5.512e+07,1.736e+07,184.777,18.2,0.000277,15275.7,15460.5
2010-01-01 04:00:00,-3.3,1,1,5,-2.759015,2.739840,9.187560,55.0,48.0,54.86,...,0.239514,0.003885,7.248e+07,5.512e+07,1.736e+07,184.777,18.3,0.000279,15359.6,15544.4
2010-01-01 05:00:00,-3.3,1,1,6,-2.762261,2.735587,9.187560,55.0,48.0,54.86,...,0.239514,0.003885,7.248e+07,5.512e+07,1.736e+07,184.777,18.3,0.000279,15359.6,15544.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2010-12-31 19:00:00,-1.9,365,12,8756,-2.732825,2.778268,9.187559,55.0,48.0,54.79,...,0.239514,0.053699,7.248e+07,5.512e+07,1.736e+07,2554,16.9,0.000257,14184.6,16738.6
2010-12-31 20:00:00,-2.2,365,12,8757,-2.736120,2.773985,9.187559,55.0,48.0,54.79,...,0.239514,0.049895,7.248e+07,5.512e+07,1.736e+07,2373.1,17.2,0.000262,14436.4,16809.4
2010-12-31 21:00:00,-2.3,365,12,8758,-2.739409,2.769705,9.187559,55.0,48.0,54.79,...,0.239514,0.006780,7.248e+07,5.512e+07,1.736e+07,322.489,17.3,0.000263,14520.3,14842.8


## Calculate User Temperature

This is the temperature that the users receive by the system. The temperature is based on their personal preferences, and the current season of the year.

In [118]:
def Tuser(fila):
    Tsh = fila.space_heating_temp
    Tdhw = fila.hot_water_temp
    month = fila.month
    
    #Las proporciones dependen de la localización
    if month==1:
        resultado= 0.98*Tsh+0.02*Tdhw
       
    elif month==2:
        resultado=0.96*Tsh+0.04*Tdhw
    
    elif month==3:
        resultado=0.94*Tsh+0.06*Tdhw
    
    elif month==4:
        resultado=0.90*Tsh+0.1*Tdhw
    
    elif month==5:
        resultado=0.75*Tsh+0.25*Tdhw
    
    elif month==6:
        resultado=0.25*Tsh+0.75*Tdhw
    
    elif month==7:
        resultado=0*Tsh+1.0*Tdhw
    
    elif month==8:
        resultado=0*Tsh+1.0*Tdhw
    
    elif month==9:
        resultado=0.1*Tsh+0.9*Tdhw
    
    elif month==10:
        resultado=0.80*Tsh+0.2*Tdhw
    
    elif month==11:
        resultado=0.95*Tsh+0.05*Tdhw
    
    elif month==12:
        resultado=0.97*Tsh+0.03*Tdhw
        
    return resultado


In [119]:
sh_consumption_ratio = Tamb_hourly["SH_consumption"] / Tamb_hourly["Total_consumption"]
dhw_consumption_ratio = Tamb_hourly["DHW_consumption"] / Tamb_hourly["Total_consumption"]

In [120]:
#Tamb_hourly["user_temp_2"] = Tamb_hourly.apply(Tuser, axis=1)
Tamb_hourly["user_temp"] = (sh_consumption_ratio * Tamb_hourly.space_heating_temp 
                            + dhw_consumption_ratio * Tamb_hourly.hot_water_temp)

Tamb_hourly

,air_temp,dayofyear,month,hourofyear,air_temp_fit,ground_temp,aquifer_temp,space_heating_temp,hot_water_temp,user_temp,...,%DHW_y,DHW_hourly_consumption_ratio,Heat_demand,SH_demand,DHW_demand,DHW_consumption,heating_degree_days,space_heating_dist,SH_consumption,Total_consumption
timestamp,,,,,,,,,,,,,,,,,,,,,
2010-01-01 01:00:00,-2.9,1,1,2,-2.749240,2.752620,9.187559,55.0,48.0,54.915,...,0.239514,0.003885,7.248e+07,5.512e+07,1.736e+07,184.777,17.9,0.000273,15023.9,15208.7
2010-01-01 02:00:00,-3.1,1,1,3,-2.752504,2.748357,9.187560,55.0,48.0,54.9159,...,0.239514,0.003885,7.248e+07,5.512e+07,1.736e+07,184.777,18.1,0.000276,15191.7,15376.5
2010-01-01 03:00:00,-3.2,1,1,4,-2.755763,2.744097,9.187560,55.0,48.0,54.9163,...,0.239514,0.003885,7.248e+07,5.512e+07,1.736e+07,184.777,18.2,0.000277,15275.7,15460.5
2010-01-01 04:00:00,-3.3,1,1,5,-2.759015,2.739840,9.187560,55.0,48.0,54.9168,...,0.239514,0.003885,7.248e+07,5.512e+07,1.736e+07,184.777,18.3,0.000279,15359.6,15544.4
2010-01-01 05:00:00,-3.3,1,1,6,-2.762261,2.735587,9.187560,55.0,48.0,54.9168,...,0.239514,0.003885,7.248e+07,5.512e+07,1.736e+07,184.777,18.3,0.000279,15359.6,15544.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2010-12-31 19:00:00,-1.9,365,12,8756,-2.732825,2.778268,9.187559,55.0,48.0,53.9319,...,0.239514,0.053699,7.248e+07,5.512e+07,1.736e+07,2554,16.9,0.000257,14184.6,16738.6
2010-12-31 20:00:00,-2.2,365,12,8757,-2.736120,2.773985,9.187559,55.0,48.0,54.0118,...,0.239514,0.049895,7.248e+07,5.512e+07,1.736e+07,2373.1,17.2,0.000262,14436.4,16809.4
2010-12-31 21:00:00,-2.3,365,12,8758,-2.739409,2.769705,9.187559,55.0,48.0,54.8479,...,0.239514,0.006780,7.248e+07,5.512e+07,1.736e+07,322.489,17.3,0.000263,14520.3,14842.8


## Calculate Sources Temperature based on Working Hours

In [121]:
dayofweeks = Tamb_hourly.index.dayofweek
hours = Tamb_hourly.index.hour

In [122]:
Tamb_hourly["source1_temp"] = Ts1 * s1_schedule[hours, dayofweeks]
Tamb_hourly["source2_temp"] = Ts2 * s2_schedule[hours, dayofweeks]
Tamb_hourly

,air_temp,dayofyear,month,hourofyear,air_temp_fit,ground_temp,aquifer_temp,space_heating_temp,hot_water_temp,user_temp,...,%DHW_y,DHW_hourly_consumption_ratio,Heat_demand,SH_demand,DHW_demand,DHW_consumption,heating_degree_days,space_heating_dist,SH_consumption,Total_consumption
timestamp,,,,,,,,,,,,,,,,,,,,,
2010-01-01 01:00:00,-2.9,1,1,2,-2.749240,2.752620,9.187559,55.0,48.0,54.915,...,0.239514,0.003885,7.248e+07,5.512e+07,1.736e+07,184.777,17.9,0.000273,15023.9,15208.7
2010-01-01 02:00:00,-3.1,1,1,3,-2.752504,2.748357,9.187560,55.0,48.0,54.9159,...,0.239514,0.003885,7.248e+07,5.512e+07,1.736e+07,184.777,18.1,0.000276,15191.7,15376.5
2010-01-01 03:00:00,-3.2,1,1,4,-2.755763,2.744097,9.187560,55.0,48.0,54.9163,...,0.239514,0.003885,7.248e+07,5.512e+07,1.736e+07,184.777,18.2,0.000277,15275.7,15460.5
2010-01-01 04:00:00,-3.3,1,1,5,-2.759015,2.739840,9.187560,55.0,48.0,54.9168,...,0.239514,0.003885,7.248e+07,5.512e+07,1.736e+07,184.777,18.3,0.000279,15359.6,15544.4
2010-01-01 05:00:00,-3.3,1,1,6,-2.762261,2.735587,9.187560,55.0,48.0,54.9168,...,0.239514,0.003885,7.248e+07,5.512e+07,1.736e+07,184.777,18.3,0.000279,15359.6,15544.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2010-12-31 19:00:00,-1.9,365,12,8756,-2.732825,2.778268,9.187559,55.0,48.0,53.9319,...,0.239514,0.053699,7.248e+07,5.512e+07,1.736e+07,2554,16.9,0.000257,14184.6,16738.6
2010-12-31 20:00:00,-2.2,365,12,8757,-2.736120,2.773985,9.187559,55.0,48.0,54.0118,...,0.239514,0.049895,7.248e+07,5.512e+07,1.736e+07,2373.1,17.2,0.000262,14436.4,16809.4
2010-12-31 21:00:00,-2.3,365,12,8758,-2.739409,2.769705,9.187559,55.0,48.0,54.8479,...,0.239514,0.006780,7.248e+07,5.512e+07,1.736e+07,322.489,17.3,0.000263,14520.3,14842.8


## Calculate Network temperature

In [123]:
def calculate_tnet(temp_s1, temp_s2, temp_aq):
    if temp_s1 == 0.0 and temp_s2 == 0.0:
        return temp_aq
    elif temp_s1 == 0.0:
        return temp_s2
    elif temp_s2 == 0.0:
        return temp_s1
    else:
        return np.mean([temp_s1, temp_s2])

In [124]:
Tamb_hourly["net_temp"] = Tamb_hourly.apply(lambda fila: calculate_tnet(fila.source1_temp, fila.source2_temp, fila.aquifer_temp), axis=1)
Tamb_hourly

,air_temp,dayofyear,month,hourofyear,air_temp_fit,ground_temp,aquifer_temp,space_heating_temp,hot_water_temp,user_temp,...,%DHW_y,DHW_hourly_consumption_ratio,Heat_demand,SH_demand,DHW_demand,DHW_consumption,heating_degree_days,space_heating_dist,SH_consumption,Total_consumption
timestamp,,,,,,,,,,,,,,,,,,,,,
2010-01-01 01:00:00,-2.9,1,1,2,-2.749240,2.752620,9.187559,55.0,48.0,54.915,...,0.239514,0.003885,7.248e+07,5.512e+07,1.736e+07,184.777,17.9,0.000273,15023.9,15208.7
2010-01-01 02:00:00,-3.1,1,1,3,-2.752504,2.748357,9.187560,55.0,48.0,54.9159,...,0.239514,0.003885,7.248e+07,5.512e+07,1.736e+07,184.777,18.1,0.000276,15191.7,15376.5
2010-01-01 03:00:00,-3.2,1,1,4,-2.755763,2.744097,9.187560,55.0,48.0,54.9163,...,0.239514,0.003885,7.248e+07,5.512e+07,1.736e+07,184.777,18.2,0.000277,15275.7,15460.5
2010-01-01 04:00:00,-3.3,1,1,5,-2.759015,2.739840,9.187560,55.0,48.0,54.9168,...,0.239514,0.003885,7.248e+07,5.512e+07,1.736e+07,184.777,18.3,0.000279,15359.6,15544.4
2010-01-01 05:00:00,-3.3,1,1,6,-2.762261,2.735587,9.187560,55.0,48.0,54.9168,...,0.239514,0.003885,7.248e+07,5.512e+07,1.736e+07,184.777,18.3,0.000279,15359.6,15544.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2010-12-31 19:00:00,-1.9,365,12,8756,-2.732825,2.778268,9.187559,55.0,48.0,53.9319,...,0.239514,0.053699,7.248e+07,5.512e+07,1.736e+07,2554,16.9,0.000257,14184.6,16738.6
2010-12-31 20:00:00,-2.2,365,12,8757,-2.736120,2.773985,9.187559,55.0,48.0,54.0118,...,0.239514,0.049895,7.248e+07,5.512e+07,1.736e+07,2373.1,17.2,0.000262,14436.4,16809.4
2010-12-31 21:00:00,-2.3,365,12,8758,-2.739409,2.769705,9.187559,55.0,48.0,54.8479,...,0.239514,0.006780,7.248e+07,5.512e+07,1.736e+07,322.489,17.3,0.000263,14520.3,14842.8


# Coefficient of performance (COP)

In [125]:
DT_hx = 2.5
n_HP = 0.53
Te_o = Tamb_hourly.net_temp - DT_evap - DT_hx
Tc_o = Tamb_hourly.user_temp

Tamb_hourly['COP']= 1-n_HP + n_HP * (Tc_o + 273.15) / (Tc_o + DT_hx - Te_o)
Tamb_hourly

,air_temp,dayofyear,month,hourofyear,air_temp_fit,ground_temp,aquifer_temp,space_heating_temp,hot_water_temp,user_temp,...,%DHW_y,DHW_hourly_consumption_ratio,Heat_demand,SH_demand,DHW_demand,DHW_consumption,heating_degree_days,space_heating_dist,SH_consumption,Total_consumption
timestamp,,,,,,,,,,,,,,,,,,,,,
2010-01-01 01:00:00,-2.9,1,1,2,-2.749240,2.752620,9.187559,55.0,48.0,54.915,...,0.239514,0.003885,7.248e+07,5.512e+07,1.736e+07,184.777,17.9,0.000273,15023.9,15208.7
2010-01-01 02:00:00,-3.1,1,1,3,-2.752504,2.748357,9.187560,55.0,48.0,54.9159,...,0.239514,0.003885,7.248e+07,5.512e+07,1.736e+07,184.777,18.1,0.000276,15191.7,15376.5
2010-01-01 03:00:00,-3.2,1,1,4,-2.755763,2.744097,9.187560,55.0,48.0,54.9163,...,0.239514,0.003885,7.248e+07,5.512e+07,1.736e+07,184.777,18.2,0.000277,15275.7,15460.5
2010-01-01 04:00:00,-3.3,1,1,5,-2.759015,2.739840,9.187560,55.0,48.0,54.9168,...,0.239514,0.003885,7.248e+07,5.512e+07,1.736e+07,184.777,18.3,0.000279,15359.6,15544.4
2010-01-01 05:00:00,-3.3,1,1,6,-2.762261,2.735587,9.187560,55.0,48.0,54.9168,...,0.239514,0.003885,7.248e+07,5.512e+07,1.736e+07,184.777,18.3,0.000279,15359.6,15544.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2010-12-31 19:00:00,-1.9,365,12,8756,-2.732825,2.778268,9.187559,55.0,48.0,53.9319,...,0.239514,0.053699,7.248e+07,5.512e+07,1.736e+07,2554,16.9,0.000257,14184.6,16738.6
2010-12-31 20:00:00,-2.2,365,12,8757,-2.736120,2.773985,9.187559,55.0,48.0,54.0118,...,0.239514,0.049895,7.248e+07,5.512e+07,1.736e+07,2373.1,17.2,0.000262,14436.4,16809.4
2010-12-31 21:00:00,-2.3,365,12,8758,-2.739409,2.769705,9.187559,55.0,48.0,54.8479,...,0.239514,0.006780,7.248e+07,5.512e+07,1.736e+07,322.489,17.3,0.000263,14520.3,14842.8


# Heat losses

In [126]:
def heat_losses(T_net,T_gr, DT_evap):
    U = 2
    T_ret = T_net - DT_evap
    HL_s = (T_net - T_gr) * U / 1000 # Heat losses supply pipe [MWh]
    HL_r = (T_ret - T_gr) * U / 1000 # Heat losses return pipe [MWh]
    
    return pd.Series([HL_s, HL_r, HL_s + HL_r] , index=['E_loss_s','E_loss_r','E_loss_tot'])


In [127]:
x = Tamb_hourly.apply(lambda fila: heat_losses(fila["net_temp"], fila["ground_temp"], DT_evap), axis=1, result_type='expand')

In [128]:
Tamb_hourly['E_loss_s'] = x['E_loss_s']
Tamb_hourly['E_loss_r'] = x['E_loss_r']
Tamb_hourly['E_loss_tot'] = x['E_loss_tot']
Tamb_hourly

,air_temp,dayofyear,month,hourofyear,air_temp_fit,ground_temp,aquifer_temp,space_heating_temp,hot_water_temp,user_temp,...,%DHW_y,DHW_hourly_consumption_ratio,Heat_demand,SH_demand,DHW_demand,DHW_consumption,heating_degree_days,space_heating_dist,SH_consumption,Total_consumption
timestamp,,,,,,,,,,,,,,,,,,,,,
2010-01-01 01:00:00,-2.9,1,1,2,-2.749240,2.752620,9.187559,55.0,48.0,54.915,...,0.239514,0.003885,7.248e+07,5.512e+07,1.736e+07,184.777,17.9,0.000273,15023.9,15208.7
2010-01-01 02:00:00,-3.1,1,1,3,-2.752504,2.748357,9.187560,55.0,48.0,54.9159,...,0.239514,0.003885,7.248e+07,5.512e+07,1.736e+07,184.777,18.1,0.000276,15191.7,15376.5
2010-01-01 03:00:00,-3.2,1,1,4,-2.755763,2.744097,9.187560,55.0,48.0,54.9163,...,0.239514,0.003885,7.248e+07,5.512e+07,1.736e+07,184.777,18.2,0.000277,15275.7,15460.5
2010-01-01 04:00:00,-3.3,1,1,5,-2.759015,2.739840,9.187560,55.0,48.0,54.9168,...,0.239514,0.003885,7.248e+07,5.512e+07,1.736e+07,184.777,18.3,0.000279,15359.6,15544.4
2010-01-01 05:00:00,-3.3,1,1,6,-2.762261,2.735587,9.187560,55.0,48.0,54.9168,...,0.239514,0.003885,7.248e+07,5.512e+07,1.736e+07,184.777,18.3,0.000279,15359.6,15544.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2010-12-31 19:00:00,-1.9,365,12,8756,-2.732825,2.778268,9.187559,55.0,48.0,53.9319,...,0.239514,0.053699,7.248e+07,5.512e+07,1.736e+07,2554,16.9,0.000257,14184.6,16738.6
2010-12-31 20:00:00,-2.2,365,12,8757,-2.736120,2.773985,9.187559,55.0,48.0,54.0118,...,0.239514,0.049895,7.248e+07,5.512e+07,1.736e+07,2373.1,17.2,0.000262,14436.4,16809.4
2010-12-31 21:00:00,-2.3,365,12,8758,-2.739409,2.769705,9.187559,55.0,48.0,54.8479,...,0.239514,0.006780,7.248e+07,5.512e+07,1.736e+07,322.489,17.3,0.000263,14520.3,14842.8


# Electricity consumption

The hourly electricity consumption by climatic zone is estimated as a function of the total thermal demand and the coefficient of performance of the systems substations:

\begin{align*}
E_{el} = \frac{E_{th}}{COP}
\end{align*}

## Save CSV version of the dataset and simulation values

In [129]:
AVAILABLE_DATASET_PATHS = {OspitalettoDataset.OSPITALETTO: ospitaletto2019.processed_dataset_path,
                           NOAA2010Dataset.MIAMI_FL: noaa2010Dataset.processed_miami_fl_dataset_path, 
                           NOAA2010Dataset.FRESNO_CA: noaa2010Dataset.processed_fresno_ca_dataset_path, 
                           NOAA2010Dataset.OLYMPIA_WA: noaa2010Dataset.processed_olympia_wa_dataset_path,
                           NOAA2010Dataset.ROCHESTER_NY: noaa2010Dataset.processed_rochester_ny_dataset_path,
                           InsPireDataset.LONDON_UK: insPireDataset.processed_london_uk_dataset_path,
                           InsPireDataset.MADRID_SPA: insPireDataset.processed_madrid_spa_dataset_path,
                           InsPireDataset.ROME_IT: insPireDataset.processed_rome_it_dataset_path,
                           InsPireDataset.STUTTGART_GER: insPireDataset.processed_stuttgart_ger_dataset_path,}

Tamb_hourly.to_csv(path_or_buf=AVAILABLE_DATASET_PATHS[dataset_to_work])

In [130]:
simulation_values_path = os.path.join(".", "data", "simulation_values.json")

with open(simulation_values_path, "w") as f:
    json.dump(simulation_values, f, sort_keys=True, indent=2)